In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import DataAgent, TacProcess
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta


path = "./data/"


In [2]:

model = load('model/best_model_180')

online180 = load('model/online180')   

train_by_step(model, 1, online180)

print(f"Trained {model}")


Trained Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                ('m', QuadraticDiscriminantAnalysis())])


In [3]:
class MockModel():
    
    def __init__(self, 
                 model
                ):
        self.model = model
        self.normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
        
        
    def fit(self, x, y):
        self.normalizer.adapt(x)
        norm_x = self.normalizer(x).numpy()
        self.model.fit(norm_x, y)
        
    def predict(self, x):
        norm_x = self.normalizer(x).numpy()
        return self.model.predict(norm_x)
        
#model = MockModel(model)

In [4]:
back = BackTest(value = 100,
                    verbose = True, 
                    sell_on_profit = True)

request_sell = lambda price: back.request_sell(price)
request_buy = lambda price: back.request_buy(price)
on_state = lambda timestamp, price: back.on_state(timestamp, price)

stock = StockAgent(
    model = model,
    request_sell = request_sell,
    request_buy = request_buy,
    on_state = on_state
)


In [5]:
start = int(datetime.timestamp((datetime.now() + timedelta( hours=-5 ))))

online180 = OnLineDataProvider(
             step = 180,
             train_keys = ["btcusd"],
             train_limit = 1000,
             val_limit = 1000,
             val_keys = [],
             val_start = 1623435037,
             train_start_list = [start]
)

online180.load_cache()

x, y = online180.load_train_data()


https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 180, 'limit': 1000, 'start': 1626062156}
Downloaded from 2021-07-12 04:54:00 to 2021-07-12 09:54:00 34281.05
Resample 2Min - TecAn ( windows 20, windows_limit 100 )
Processing 100 of btcusd


  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
from bitstamp import *

on_new_data = lambda x: print(x)
on_new_data = lambda x: stock.on_x(x)

on_state = lambda timestamp, price, buy, sell: print("{} {} {} {}".format(timestamp, price, buy, sell))
on_state = lambda timestamp, price, buy, sell: stock.on_new_state(timestamp, price, buy, sell)

agent = DataAgent(
    taProc = TacProcess(), 
    tec = TecAn(windows = 20, windows_limit = 100),
    resample = '3Min',
    on_state = on_state,
    on_new_data = on_new_data
)

for yy in y:
    #print(yy)
    agent.taProc.add_tacs_realtime([], yy, 0.0, agent.tec)


TIMESTAMP_KEY = "timestamp"

raw_data_live = []


class RawStateDownloader(LiveBitstamp):
    
    def __init__(self, on_raw_data = lambda raw: print(raw)):
        self.trade = {}
        self.on_new_data_count = 0
        self.on_raw_data = on_raw_data
                
    def process(self, raw):
        raw_data_live.append(raw)
        self.on_raw_data(raw)
        timestamp = raw[TIMESTAMP_KEY]
        time = pd.to_datetime(timestamp, unit='s')
        #print(f'{time}: {agent.on_new_data_count} {len(raw_data_live)}')
        

Resample 3Min - TecAn ( windows 20, windows_limit 100 )


In [7]:

on_raw_data = lambda raw: agent.on_new_raw_data(raw)


live = RawStateDownloader(on_raw_data = on_raw_data)


In [ ]:
back.reset()

bt = Bitstamp(live, currency = "btcusd")

bt.connect()

Bitstamp Websocket Opened.
Reading form live_trades_btcusd and order_book_btcusd


In [ ]:
back.report()